In [1]:
from dataclasses import dataclass
import math
import pandas as pd

PAR = 100.0

def price_zero(maturity: int, y: float) -> float:
    return PAR / ((1.0 + y) ** maturity)

def price_coupon_bond(coupon_rate: float, maturity: int, y: float) -> float:
    """coupon_rate quoted in %, annual coupon; returns price per 100 par."""
    c = coupon_rate / 100.0 * PAR
    pv_coupons = sum(c / ((1.0 + y) ** t) for t in range(1, maturity + 1))
    pv_principal = PAR / ((1.0 + y) ** maturity)
    return pv_coupons + pv_principal

def liability_pv_and_duration(y: float):
    """Return PV and Macaulay duration (years) for liability at yield y."""
    pv30 = 1_000_000 / ((1.0 + y) ** 30)
    pv31 = 2_000_000 / ((1.0 + y) ** 31)
    pv = pv30 + pv31
    D_mac = (30 * pv30 + 31 * pv31) / pv
    return pv, D_mac, pv30, pv31

def replicate_liability(y: float):
    """Exact synthetic replication using A (31y zero), B (30y 4%), C (30y 6%)."""
    # prices
    PA = price_zero(31, y)
    PB = price_coupon_bond(4.0, 30, y)
    PC = price_coupon_bond(6.0, 30, y)

    # Building block: 3*B - 2*C = $100 only at t=30
    blocks = 1_000_000 / PAR  # number of 100-par "blocks" for $1M at t=30
    nB = 3.0 * blocks
    nC = -2.0 * blocks
    nA = 2_000_000 / PAR  # $2M at t=31 via 31y zero

    # Market value
    mv = nA * PA + nB * PB + nC * PC

    df = pd.DataFrame({
        "Bond": ["A (31y zero)", "B (30y 4%)", "C (30y 6%)"],
        "Units (par=100)": [nA, nB, nC],
        "Price per 100": [PA, PB, PC],
        "Market Value": [nA * PA, nB * PB, nC * PC]
    })
    return df, mv

def duration_hedge_liability(y: float):
    """Match PV and Macaulay duration using 10y and 15y zeros; return table and (n10,n15)."""
    pv, D_mac, *_ = liability_pv_and_duration(y)
    p10 = price_zero(10, y)
    p15 = price_zero(15, y)

    # Solve:
    # n10*p10 + n15*p15 = pv
    # 10*n10*p10 + 15*n15*p15 = D_mac * pv
    import numpy as np
    A = np.array([[p10, p15],
                  [10 * p10, 15 * p15]], dtype=float)
    b = np.array([pv, D_mac * pv], dtype=float)
    n10, n15 = np.linalg.solve(A, b)

    df = pd.DataFrame({
        "Zero": ["10y zero", "15y zero"],
        "Units (par=100)": [n10, n15],
        "Price per 100": [p10, p15],
        "Market Value": [n10 * p10, n15 * p15],
        "Macaulay Duration (y)": [10, 15]
    })
    return df, (n10, n15), pv, D_mac

def what_if_rebalance(y_new: float):
    """Recompute PV/duration at new rate and solve new (n10,n15) for exact match."""
    pv, D_mac, *_ = liability_pv_and_duration(y_new)
    p10 = price_zero(10, y_new)
    p15 = price_zero(15, y_new)
    import numpy as np
    A = np.array([[p10, p15],
                  [10 * p10, 15 * p15]], dtype=float)
    b = np.array([pv, D_mac * pv], dtype=float)
    n10, n15 = np.linalg.solve(A, b)
    return {"y_new": y_new, "pv": pv, "D_mac": D_mac, "n10": n10, "n15": n15, "p10": p10, "p15": p15}

if __name__ == "__main__":
    y = 0.05  # base flat yield

    # (a) Exact replication
    df_rep, mv_rep = replicate_liability(y)
    print("=== (a) Synthetic replication with A (31y ZC), B (30y 4%), C (30y 6%) ===")
    print(df_rep.to_string(index=False))
    print(f"Total market value of hedging portfolio: ${mv_rep:,.2f}")

    pv, D_mac, pv30, pv31 = liability_pv_and_duration(y)
    print("\nLiability PV and duration at 5%:")
    print(f"PV_30 = ${pv30:,.2f}, PV_31 = ${pv31:,.2f}, Total PV = ${pv:,.2f}, D_mac = {D_mac:.8f} yrs")

    # (b) Duration-based hedge with 10y & 15y zeros
    df_dur, (n10, n15), pv_l, D_l = duration_hedge_liability(y)
    print("\n=== (b) Duration hedge with 10y & 15y zeros ===")
    print(df_dur.to_string(index=False))
    print(f"PV matched: ${pv_l:,.2f}, D_mac matched: {D_l:.8f} yrs")
    print(f"Units: n10={n10:.6f} (short if negative), n15={n15:.6f}")

    # (c) What-if: rates fall to 4.8% (rebalance illustration)
    res = what_if_rebalance(0.048)
    print("\n=== (c) If rates fall to 4.8%, rebalance illustration (exact PV & duration match) ===")
    print(f"New PV = ${res['pv']:,.2f}, New D_mac = {res['D_mac']:.8f} yrs")
    print(f"Prices: P10={res['p10']:.6f}, P15={res['p15']:.6f}")
    print(f"New units: n10={res['n10']:.6f}, n15={res['n15']:.6f}")

=== (a) Synthetic replication with A (31y ZC), B (30y 4%), C (30y 6%) ===
        Bond  Units (par=100)  Price per 100  Market Value
A (31y zero)          20000.0      22.035947  4.407189e+05
  B (30y 4%)          30000.0      84.627549  2.538826e+06
  C (30y 6%)         -20000.0     115.372451 -2.307449e+06
Total market value of hedging portfolio: $672,096.40

Liability PV and duration at 5%:
PV_30 = $231,377.45, PV_31 = $440,718.95, Total PV = $672,096.40, D_mac = 30.65573770 yrs

=== (b) Duration hedge with 10y & 15y zeros ===
    Zero  Units (par=100)  Price per 100  Market Value  Macaulay Duration (y)
10y zero    -34278.995823      61.391325 -2.104433e+06                     10
15y zero     57722.051771      48.101710  2.776529e+06                     15
PV matched: $672,096.40, D_mac matched: 30.65573770 yrs
Units: n10=-34278.995823 (short if negative), n15=57722.051771

=== (c) If rates fall to 4.8%, rebalance illustration (exact PV & duration match) ===
New PV = $712,549.76, Ne